## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
#city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

#city_data_to_load = os.path.join("Weather_Database", "WeatherPy_database.csv")
city_data_to_load = os.path.join("..","Weather_Database", "WeatherPy_database.csv")
city_data_df = pd.read_csv(city_data_to_load)

city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Yellowknife,CA,62.4560,-114.3525,-21.59,75,100,4.61,overcast clouds
1,1,Bethel,US,41.3712,-73.4140,32.05,82,100,9.22,light snow
2,2,Road Town,VG,18.4167,-64.6167,80.69,78,40,13.80,scattered clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,50.11,57,20,57.53,light rain
4,4,Mataura,NZ,-46.1927,168.8643,43.48,99,31,2.51,scattered clouds


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
2,2,Road Town,VG,18.4167,-64.6167,80.69,78,40,13.80,scattered clouds
5,5,Pisco,PE,-13.7000,-76.2167,75.25,60,0,6.91,clear sky
10,10,Butaritari,KI,3.0707,172.7902,82.53,70,95,11.61,overcast clouds
11,11,Vao,NC,-22.6667,167.4833,78.89,85,100,5.68,overcast clouds
13,13,Avarua,CK,-21.2078,-159.7750,73.45,100,20,2.30,few clouds
14,14,Atuona,PF,-9.8000,-139.0333,77.38,78,33,12.62,light rain
17,17,Springbok,ZA,-29.6643,17.8865,84.34,31,46,16.60,scattered clouds
19,19,Rikitea,PF,-23.1203,-134.9692,78.55,70,8,10.89,clear sky
20,20,Otacilio Costa,BR,-27.4831,-50.1219,87.80,33,27,2.33,scattered clouds
23,23,Poum,NC,-20.2333,164.0167,80.58,78,100,14.92,overcast clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        9
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [34]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [33]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Road Town,VG,80.69,scattered clouds,18.4167,-64.6167,
5,Pisco,PE,75.25,clear sky,-13.7000,-76.2167,
10,Butaritari,KI,82.53,overcast clouds,3.0707,172.7902,
11,Vao,NC,78.89,overcast clouds,-22.6667,167.4833,
13,Avarua,CK,73.45,few clouds,-21.2078,-159.7750,
14,Atuona,PF,77.38,light rain,-9.8000,-139.0333,
17,Springbok,ZA,84.34,scattered clouds,-29.6643,17.8865,
19,Rikitea,PF,78.55,clear sky,-23.1203,-134.9692,
20,Otacilio Costa,BR,87.80,scattered clouds,-27.4831,-50.1219,
23,Poum,NC,80.58,overcast clouds,-20.2333,164.0167,


In [36]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   
# 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
   
    results=hotels['results']
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [54]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= ""]
clean_hotel_df.count()

City           231
Country        231
Max Temp       231
Description    231
Lat            231
Lng            231
Hotel Name     231
dtype: int64

In [55]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [59]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [65]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))